# Data Wrangling
Notes on data wrangling in Pandas

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [2]:
# Combining and merging data sets
## pandas.merge --> connect rows in DataFrames,
### based one one or more keys; SQL join operations
## pandas.concat --> glues or stacks objs together
### along an axis
## combine_first --> instance method enables splicing overlapping
### data to fill in missing values in one object with values
### from another object

In [4]:
# Database-style DataFrame Merges
# merge and join operations combine data sets by linking rows
# using one or more keys.
# these are central operations to relational DBs
# Merge function in pandas is the entry point for using these algos on your data
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                'data1': range(7)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [6]:
df2 = DataFrame({'key': ['a', 'b', 'd'],
                'data2': range(3)})
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [7]:
# this a many-to-one merge. 
# df1 has multiple rows labeled a and b,
# but df2 has only one row for value in the key column
# let's call these objects
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [8]:
# if not specified, merge uses the overlapping column names as keys
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [10]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                'data1': range(7)})
df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [12]:
df4 = DataFrame({'rkey': ['a', 'b', 'd'],
                'data2': range(3)})
df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [13]:
# if column names are different in each object,
# you can specify them separately
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [14]:
# c and d values and associated data are missing from result, above
# by default, merge does an 'inner' join
# the keys in the result are the intersection
# other options: left, right, and outer
# OUTER join takes the UNION of keys, which is like performing
# BOTH left and right joins
pd.merge(df1,df2,how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [16]:
# many to many merges -> behavior is well defined, but not very intuitive
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                'data1': range(6)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [18]:
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                'data2': range(5)})
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [20]:
pd.merge(df1,df2,on='key',how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [21]:
pd.merge(df1,df2,on='key')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [22]:
pd.merge(df1,df2,on='key',how='right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,5.0,b,1
3,0.0,b,3
4,1.0,b,3
5,5.0,b,3
6,2.0,a,0
7,4.0,a,0
8,2.0,a,2
9,4.0,a,2


In [23]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [25]:
# can merge with multiple keys by passing a list of column names
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                'key2': ['one', 'two', 'one'],
                'lval': [1, 2, 3]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [26]:
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                'key2': ['one', 'one', 'one', 'two'],
                'rval': [4, 5, 6, 7]})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [27]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [29]:
# suffix options specify how to handle overlapping names
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [30]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [32]:
# merging on index
# in this case, left_index=True, or right_index=True (or both)
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                    'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [34]:
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

,group_val
a,3.5
b,7.0


In [35]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [36]:
# since default method is to intersect the join keys,
# you can instead form the union of them with outer join
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [39]:
# with hierarchically indexed data, it's more complicated
lefth = DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
            'key2': [2000, 2001, 2002, 2001, 2002],
            'data': np.arange(5.)})
lefth

,data,key1,key2
0,0.0,Ohio,2000
1,1.0,Ohio,2001
2,2.0,Ohio,2002
3,3.0,Nevada,2001
4,4.0,Nevada,2002


In [41]:
righth = DataFrame(np.arange(12).reshape((6, 2)),
    index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
    [2001, 2000, 2000, 2000, 2001, 2002]],
    columns=['event1', 'event2'])
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [43]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,data,key1,key2,event1,event2
0,0.0,Ohio,2000,4,5
0,0.0,Ohio,2000,6,7
1,1.0,Ohio,2001,8,9
2,2.0,Ohio,2002,10,11
3,3.0,Nevada,2001,0,1


In [45]:
left2 = DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],
        columns=['Ohio', 'Nevada'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [47]:
right2 = DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
    index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [48]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [49]:
# JOIN is more convenient for merging by index
# can also use for combining many DataFrame objects having same,
# or similar indices, but not overlapping columns
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [51]:
# also supports joining the index of the passed DataFrame
# on one of the columns of the calling DataFrame
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [53]:
# pass a list of dataframes, as an alternative to concat
another = DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
        index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [54]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [55]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


In [57]:
# concatenating along an axis
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [58]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [60]:
# numpy has this function
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [62]:
# assume 3 series with no overlap
s1 = Series([0, 1], index=['a', 'b'])
s1

a    0
b    1
dtype: int64

In [63]:
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s2

c    2
d    3
e    4
dtype: int64

In [65]:
s3 = Series([5, 6], index=['f', 'g'])
s3

f    5
g    6
dtype: int64

In [66]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [67]:
# by default, concat works along axis=0, producing another Series
# if axis=1, yields a DataFrame (axis=1 targets cols)
pd.concat([s1,s2,s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [69]:
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [70]:
pd.concat([s1, s4], axis=1) 

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [71]:
# intersect by passing join=inner
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


In [72]:
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


In [73]:
# Reshaping and Pivoting
## primary actions: stack --> rotate cols to rows
##   and unstack --> pivots from rows into columns
data = DataFrame(np.arange(6).reshape((2, 3)),
    index=pd.Index(['Ohio', 'Colorado'], name='state'),
     columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [75]:
# using stack pivots the cols into rows, producing a series
result = data.stack()
result
# pulls the number column down into indices

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [76]:
# innermost level is stacked, by default
# a different level can be stacked by passing a number or name
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [77]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [79]:
# can introuce missing data if all the values aren't found in each subgroup
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s1

a    0
b    1
c    2
d    3
dtype: int64

In [80]:
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])
s2

c    4
d    5
e    6
dtype: int64

In [82]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [83]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [84]:
# stacking filters out missing data by default, so ther operation is einvertible
data2.unstack().stack() 

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [85]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [86]:
# unstacked level in DataFrame becomes lowest level in result
df = DataFrame({'left': result, 'right': result + 5},
    columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [87]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [88]:
df.unstack('state').stack('side')

state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

In [ ]:
# use PIVOT to move mixed items --> from rows to columns
# data ->
# date   | item | value
# tstamp    i1     xxxx
# tstamp    i2     xxxx
# tstamp    i3     xxxx

# to 

# data.pivot('date', 'item', 'value')
# date    | i1 | i2 | i3 
# tstamp   xxx  xxx  xxx

# by omitting last value, we get a dataframe with hierarchical columns
# pivot is a shortcut for creating a hierarchical index
# using set_index and reshaping with unstack

In [90]:
# Data Transformation
## rm duplicates
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4,
                   'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [92]:
# tells us whether each row is duplicated, or not
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [93]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


In [95]:
# mapping to transform
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami',
        'corned beef', 'Bacon', 'pastrami', 'honey ham',
        'nova lox'],
        'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [96]:
meat_to_animal = {
 'bacon': 'pig',
 'pulled pork': 'pig',
 'pastrami': 'cow',
 'corned beef': 'cow',
 'honey ham': 'pig',
 'nova lox': 'salmon'
}

In [98]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [99]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

In [100]:
# access the index of a DataFrame with .index()
data = DataFrame(np.arange(12).reshape((3, 4)),
        index=['Ohio', 'Colorado', 'New York'],
        columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [101]:
data.index = data.index.map(str.upper)
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [104]:
# discretizing and binning
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins) # use the cut function for this
cats # cats, as in categoies

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, object): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [106]:
cats.labels

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:1: FutureWarning: 'labels' is deprecated. Use 'codes' instead
  if __name__ == '__main__':


array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [108]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

In [109]:
# can also chop data uniformly
data = np.random.rand(20)
pd.cut(data, 4, precision=2)

[(0.72, 0.96], (0.24, 0.48], (0.48, 0.72], (0.72, 0.96], (0.48, 0.72], ..., (0.0031, 0.24], (0.72, 0.96], (0.0031, 0.24], (0.24, 0.48], (0.0031, 0.24]]
Length: 20
Categories (4, object): [(0.0031, 0.24] < (0.24, 0.48] < (0.48, 0.72] < (0.72, 0.96]]

In [112]:
# can bin based on quantiles, with qcut
data = np.random.randn(1000) # Normally distributed
cats = pd.qcut(data, 4) # Cut into quartiles
cats

[(-0.644, -0.0561], [-2.879, -0.644], (-0.644, -0.0561], (-0.644, -0.0561], (0.639, 3.391], ..., [-2.879, -0.644], [-2.879, -0.644], [-2.879, -0.644], (-0.0561, 0.639], (0.639, 3.391]]
Length: 1000
Categories (4, object): [[-2.879, -0.644] < (-0.644, -0.0561] < (-0.0561, 0.639] < (0.639, 3.391]]

In [113]:
pd.value_counts(cats)

(0.639, 3.391]       250
(-0.0561, 0.639]     250
(-0.644, -0.0561]    250
[-2.879, -0.644]     250
dtype: int64

In [114]:
 pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-1.327, -0.0561], (-1.327, -0.0561], (-1.327, -0.0561], (-1.327, -0.0561], (-0.0561, 1.288], ..., (-1.327, -0.0561], [-2.879, -1.327], [-2.879, -1.327], (-0.0561, 1.288], (1.288, 3.391]]
Length: 1000
Categories (4, object): [[-2.879, -1.327] < (-1.327, -0.0561] < (-0.0561, 1.288] < (1.288, 3.391]]

In [118]:
# outliers
data = DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.001790,-0.045011,0.014383,0.020419
std,0.980889,0.984121,1.016329,1.027081
min,-2.944933,-3.927477,-2.967843,-3.557374
25%,-0.660975,-0.723895,-0.663220,-0.654969
50%,-0.014137,-0.053689,0.032044,0.009570
75%,0.686289,0.628575,0.714978,0.695544
max,3.383302,3.317278,3.057070,3.707289


In [119]:
col = data[3]

In [120]:
col[np.abs(col) > 3]

206   -3.178613
248   -3.383857
707   -3.185163
712    3.707289
816   -3.006134
858    3.184161
881    3.161089
980   -3.557374
Name: 3, dtype: float64

In [121]:
# to select all the ROWS exceeding 3 or -3,
# you can use the any() method
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
206,-0.924157,0.042871,0.603923,-3.178613
248,1.113160,-1.585073,0.981738,-3.383857
250,3.383302,1.476443,0.743273,1.698929
518,2.086806,3.317278,1.232155,-0.834084
707,-0.826837,-0.311621,-0.431009,-3.185163
712,1.268598,-0.451957,-0.152763,3.707289
786,0.483294,-0.080721,3.057070,-2.451735
816,0.328246,1.322409,0.837147,-3.006134
828,-1.033748,-3.927477,0.889890,0.964973
858,-0.169620,1.061809,0.914631,3.184161


In [124]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe() # np.sign(...) returns an array of 1 and -1
                # depending sign of the values

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.002173,-0.044458,0.014326,0.020678
std,0.979639,0.979674,1.016159,1.019610
min,-2.944933,-3.000000,-2.967843,-3.000000
25%,-0.660975,-0.723895,-0.663220,-0.654969
50%,-0.014137,-0.053689,0.032044,0.009570
75%,0.686289,0.628575,0.714978,0.695544
max,3.000000,3.000000,3.000000,3.000000
